<a href="https://colab.research.google.com/github/cwangf00d/cs187-section/blob/master/Cindy_Section_1_Optimization_using_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS 187 Section 1: Optimization using PyTorch

URL: https://bit.ly/section1pytorch

Partly adapted from [this tutorial](https://pytorch.org/tutorials/beginner/pytorch_with_examples.html).

In our labs, you have seen numerous examples of tensor operations in PyTorch. However, PyTorch is most widely known as a toolkit for optimizing neural networks. In this section, we will learn how to use PyTorch to optimize an objective function. We will start from low level PyTorch tensor operations, gradually wrapping low-level operations using high-level classes/functions to simplify the code. By the end of this section, you should be able to understand the concepts of parameters, gradients, and the optimization process. This will be the foudation for implementing neural networks with much more parameters in future labs.

Goals:

1. Understand what is computation graph and how automatic differentiation works.
2. Implement automatic differentiation in PyTorch.
3. Implement optimization using high-level PyTorch abstractions `nn.Module` and `optim`.


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import torch
import torch.nn as nn

In [ ]:
# Set up plotting
plt.style.use('tableau-colorblind10')

## Gradient-Based Optimization

Consider the following problem:

$$
\min_x {(x-2)}^2.
$$

Let's pretend that we don't know how to solve this problem analytically. Instead, we will use this problem to demonstrate gradient-based optimization. We will define ${(x-2)}^2$ as our _loss function_: $L(x) = {(x-2)}^2$. We can visualize this loss function.

In [ ]:
def loss(x):
  return (x - 2) ** 2

x_vals = torch.linspace(-6, 6, 100)
y_vals = loss(x_vals)

plt.plot(x_vals, y_vals)
plt.xlabel('x')
plt.ylabel('loss')

In gradient-based optimization, we first initialize $x$ to be a random value, and then we perform the following steps:

1. Find the gradient of the loss function, the direction in which it is increasing fastest.
2. Take a step in the opposite direction.
3. Repeat.


## Manual Differentiation

For this particular problem, we can manually calculate the gradient of the loss function:

\begin{align}
\frac{\partial L(x)}{\partial x} & = \frac{\partial L(x)}{\partial (x-2)} \cdot \frac{\partial (x-2)}{\partial x}\\
&= 2(x-2) \cdot 1\\
& = 2(x-2)
\end{align}

Based on this calculation, we can implement a gradient-based optimization algorithm.

In [ ]:
def manual_grad(x):
  return 2 * (x - 2)

First, we initialize $x$ to be, say, -4. We visualize both the initial value of $x$ and the curve of the loss function.

In [ ]:
x_init = -4.
x = torch.tensor(x_init)

plt.plot(x_vals, y_vals, '-', label='$(x-2)^2$')
plt.plot(x, loss(x), 'o', label='x')
plt.xlabel('x')
plt.ylabel('loss')
plt.legend()

Next, we take a step in the opposite direction. We will update $x$ as

$$
x \leftarrow x - \eta \frac{\partial loss(x)}{\partial x},
$$
where $\eta$ is the learning rate. We'll use a learning rate of 0.1 for now.

In [ ]:
learning_rate = 0.1

In [ ]:
plt.plot(x_vals, y_vals, '-', label='$(x-2)^2$')
plt.plot(x, loss(x), 'o', label='old x')

update = - learning_rate * manual_grad(x)
x_new = x + update

plt.arrow(x, loss(x), update, 0, head_width=1, head_length=0.2, length_includes_head=True)
plt.axvline(x_new, ls='--', color='k')

x = x_new

plt.plot(x, loss(x), 'o', label='new x')
plt.xlabel('x')
plt.ylabel('loss')
plt.legend()

Let's repeat this process multiple times.

In [ ]:
from matplotlib import rc
rc('animation', html='jshtml')

all_xs = []
all_ys = []
all_updates = []

num_iters = 20
learning_rate = 0.1

def loss(x):
  return (x - 2) ** 2

# Initialize x
x_init = -4.
x = torch.tensor(x_init)

for iter in range(num_iters):
  all_xs.append(x.item())
  L = loss(x)
  all_ys.append(L.item())
  print (f'iter: {iter}, loss: {L}, x: {x}')
  all_updates.append((- learning_rate * manual_grad(x)).item())
  x.data += - learning_rate * manual_grad(x)
all_xs.append(x.item())
all_ys.append((loss(x)).item())

We can visualize this process.

In [ ]:
def animate(all_xs, all_updates):
  fig, ax = plt.subplots()
  fig.set_tight_layout(True)

  line, = ax.plot(x_vals, y_vals, 'b-', label='$(x-2)^2$')
  xold, = ax.plot(all_xs[0], all_ys[0], 'o', label='old x')

  vline = ax.axvline(all_xs[1], ls='--', color='k')

  xnew, = ax.plot(all_xs[1], all_ys[1], 'o', label='new x')
  ax.legend()
  global patch
  patch = None
  def update(i):
    global patch
    if patch in ax.patches:
      ax.patches.remove(patch) 
    num_updates = i//2
    if i % 2 == 0:
      label = f'update {num_updates} gradient computation'
      xold.set_visible(True)
      xnew.set_visible(False)
      vline.set_visible(False)
      
      patch = plt.Arrow(all_xs[num_updates], all_ys[num_updates], all_updates[num_updates], 0, width=2, color='k')
      ax.add_patch(patch)
    else:
      label = f'update {num_updates} finished'

      xold.set_visible(False)
      xnew.set_visible(True)
      vline.set_visible(True)

      vline.set_xdata([all_xs[num_updates+1], all_xs[num_updates+1]])
    
      xnew.set_xdata([all_xs[num_updates+1],])
      xnew.set_ydata([all_ys[num_updates+1],])

      xold.set_xdata([all_xs[num_updates+1],])
      xold.set_ydata([all_ys[num_updates+1],])
    ax.set_xlabel(label)
    return line, ax

  anim = FuncAnimation(fig, update, frames=range(num_iters*2), interval=1500)
  return anim

animate(all_xs, all_updates)

## Automatic Differentiation

While we can manually derive the gradient for a simple loss function with a single parameter, it would be tedious, if at all possible, to derive the gradient for a complex loss function with millions of parameters. As indicated by its name, *automatic differentiation* alleviates us from such burdens by automatically computing the gradients of a loss function with respect to its inputs.

Let's jump to an example of how to compute gradients in PyTorch. In order to enable automatic differentiation, we first need to wrap a tensor in a `torch.nn.Parameter` object.

In [ ]:
x_init = -4.              # initial parameter value
x = torch.tensor(x_init)  # tensorize it
x = nn.Parameter(x)       # declare it to be a parameter to optimize

## We build the loss function from its one parameter
z = x - 2
z.retain_grad()           # make sure to compute gradient here
L = z ** 2                # the loss 

## and compute the gradients of the loss `L`
L.backward()

## Now, we can examine the gradients
print (f'Automatic differentiation result: {x.grad}, manual differentiation result: {manual_grad(x)}')

Voila! How does it work? To understand how automatic differentiation works, we need to first understand the concept of a computation graph.

### Computation Graph

Let's take a look at what underlying data structure PyTorch uses to store a tensor. We will use a helper function to visualize the underlying data structure.



In [ ]:
!wget -q  https://raw.githubusercontent.com/nlp-course/data/master/scripts/makedot.py

In [ ]:
from makedot import make_dot
make_dot(L, params={'x': x, 'z': z, 'L':L})

Even for this basic computation, PyTorch creates an internal computation graph. Starting from $x$, it first applies a subtraction function `Sub` to compute $x-2$, and then applies a power function `Pow` with $x-2$ as input to compute ${(x-2)}^2$, which is the output $L$.

**Student Question:** Looking at this computation graph, why can we compute $\frac{\partial L}{\partial x}$ by simplying calling `L.backward()` (without mentioning `x` at all)?

As another demonstration of the computation graph, we can find the parent and the grandparent of the output node `l`. Note that `z` is an intermediate result instead of a variable, so there's no `parent.variable`.

In [ ]:
print (f"Number of parents of l: {len(L.grad_fn.next_functions)}")

parent = L.grad_fn.next_functions[0][0]

print (f"id of parent: {id(parent)}, id of z's grad_fn: {id(z.grad_fn)}")

grandparent = parent.next_functions[0][0]

print (f"id of grandparent: {id(grandparent.variable)}, id of x: {id(x)}")

Now that we can understand why it is possible to compute $\frac{\partial L}{\partial x}$ by simplying calling `L.backward()`, the next question is how does automatic differentiation work. The answer is the *chain rule* for differentiation, as illustrated below.

When we call `L.backward()`, it will first compute the gradients with respect to `L`'s parents:

<img src="https://raw.githubusercontent.com/nlp-course/data/master/img/section1_bp1_new.png" width=150 />



In [ ]:
print (z.grad)

Then grandparents using the chain rule:

<img src="https://raw.githubusercontent.com/nlp-course/data/master/img/section1_bp2_new.png" width=150 />

In summary, PyTorch creates a computation graph during the forward pass, and during the backward pass, it backpropagates gradients from the output node all the way back to the input nodes.

Now, let's try a more complex computation graph with multiple input nodes.

In [ ]:
a = torch.tensor(3.)
b = torch.tensor(4.)
c = torch.tensor(5.)

a = nn.Parameter(a)
b = nn.Parameter(b)
c = nn.Parameter(c)

L = ((a + b) * c) + c
make_dot(L, params={'a': a, 'b': b, 'c': c, 'L': L})

Similarly, we can compute $\frac{\partial L}{\partial a}$, $\frac{\partial L}{\partial b}$, and $\frac{\partial L}{\partial c}$.

<img src="https://raw.githubusercontent.com/nlp-course/data/master/img/section1_bp21_new.png" width=300 />

When we call `L.backward()`, it will first backpropagate gradients to the parent nodes of `L`:

<img src="https://raw.githubusercontent.com/nlp-course/data/master/img/section1_bp22_new.png" width=300 />

then grandparents:

<img src="https://raw.githubusercontent.com/nlp-course/data/master/img/section1_bp23_new.png" width=300 />

and then great grandparents:

<img src="https://raw.githubusercontent.com/nlp-course/data/master/img/section1_bp24_new.png" width=300 />

In [ ]:
L.backward()
print (f"dL/da: {a.grad}")
print (f"dL/db: {b.grad}")
print (f"dL/dc: {c.grad}")

### Gradient Accumulation

Note that a node in the computation graph might have multiple children (in English, that variable was used multiple times in the function), such as `c` in the above example. In such cases, we need to accumulate gradients coming from different children. Therefore, it's the default behavior of PyTorch, and unless we explicitly set gradients of a tensor to be zero, they will accumulate:

In [ ]:
a = torch.tensor(3.)
b = torch.tensor(4.)
c = torch.tensor(5.)

a = nn.Parameter(a)
b = nn.Parameter(b)
c = nn.Parameter(c)

L = ((a + b) * c) + c
L.backward()

print (f"first backward  dL/da: {a.grad}")
print (f"first backward  dL/db: {b.grad}")
print (f"first backward  dL/dc: {c.grad}\n")

L = ((a + b) * c) + c
L.backward()
print (f"second backward dL/da: {a.grad}")
print (f"second backward dL/db: {b.grad}")
print (f"second backward dL/dc: {c.grad}")

To set the gradient to zero, we use `a.grad.fill_(0)` since `a.grad` is also a tensor.

In [ ]:
a.grad.fill_(0)
b.grad.fill_(0)
c.grad.fill_(0)
L = ((a + b) * c) + c
L.backward()

print (f"first backward  dL/da: {a.grad}")
print (f"first backward  dL/db: {b.grad}")
print (f"first backward  dL/dc: {c.grad}\n")

a.grad.fill_(0)
b.grad.fill_(0)
c.grad.fill_(0)
L = ((a + b) * c) + c
L.backward()
print (f"second backward dL/da: {a.grad}")
print (f"second backward dL/db: {b.grad}")
print (f"second backward dL/dc: {c.grad}")

### `torch.no_grad`

You can disable gradient computation using context `torch.no_grad`:

In [ ]:
x_init = -4.              # initial parameter value
x = torch.tensor(x_init)  # tensorize it
x = nn.Parameter(x)       # declare it to be a parameter to optimize

## We build the loss function from its one parameter
with torch.no_grad():
  z = x - 2
  L = z ** 2                # the loss 

## and compute the gradients of the loss `L`
L.backward()

Why did it fail? What's the new computation graph?

In [ ]:
make_dot(L, params={'x': x, 'z': z, 'L':L})

**Student Question:** In what scenarios would we want to disable gradient computation?

### Optimization

Now we can perform the optimization process using PyTorch's automatic differentiation.

In [ ]:
x_init = -4.
x = torch.tensor(x_init)
x = nn.Parameter(x)

In [ ]:
from matplotlib import rc
rc('animation', html='jshtml')

all_xs = []
all_ys = []
all_updates = []

num_iters = 20
learning_rate = 0.1

def loss(x):
  return (x - 2) ** 2

# Initialize x
x_init = -4.
x = torch.tensor(x_init)

for iter in range(num_iters):
  x = nn.Parameter(x)
  all_xs.append(x.item())
  L = loss(x)
  all_ys.append(L.item())
  print (f'iter: {iter}, loss: {L}, x: {x}')
  L.backward()
  all_updates.append((- learning_rate * x.grad).item())
  x.data += - learning_rate * x.grad
  x.grad.fill_(0)
all_xs.append(x.item())
all_ys.append((loss(x)).item())

In [ ]:
animate(all_xs, all_updates)

## Scaling Up: PyTorch `nn.Module` and `optim`

We have seen how to use PyTorch automatic differentiation to optimize a few parameters. However, what if there are millions of parameters? Do we need to write a few million updates in the form of $x = x - \eta \frac{\partial loss}{\partial x}$? PyTorch provides high-level abstractions to make it much easier to scale up the number of parameters (and the complexity of the function).



Before we start, let's recap optimization using low-level PyTorch functions.

In [ ]:
num_iters = 20
learning_rate = 0.1

def loss(x):
  return (x - 2) ** 2

# Initialize x
x_init = -4.
x = torch.tensor(x_init)

for iter in range(num_iters):
  x = nn.Parameter(x)
  L = loss(x)
  print (f'iter: {iter}, loss: {L}, x: {x}')
  L.backward()
  x.data += - learning_rate * x.grad
  x.grad.fill_(0)

### `nn.Module` class

To use PyTorch's high-level abstractions, we first need to wrap the definition of the forward pass in subclass of a `nn.Module` class.

In [ ]:
class Model(nn.Module):
  # initializer, defines parameters
  def __init__(self):
    super().__init__()
    x_init = -4.
    x = torch.tensor(x_init)
    self.x = nn.Parameter(x)

  # defines the forward computation
  def forward(self):
    return (self.x - 2) ** 2

We can instantiate a `Model` object. By calling `.parameters()`, we will see a list of parameters contained within it. By calling its `forward` function, we can compute the loss as well. Note that the `forward` function does not take any input here, but in real problems it usually does.

In [ ]:
model = Model()
parameters = list(model.parameters())
print (f'The parameters of model: {parameters}')
L = model() # calling model() is the same as calling model.forward()
print (f'Loss value: {L}')

`nn.Module` also provides a `named_parameters` function, which returns the names of each parameter as well.

In [ ]:
print (list(model.named_parameters()))

We can rewrite the optimization process using this new class.

In [ ]:
num_iters = 20
learning_rate = 0.1

# Initialize x
model = Model()

for iter in range(num_iters):
  L = model()
  print (f'iter: {iter}, loss: {L}, x: {model.x}')
  L.backward()
  model.x.data += - learning_rate * model.x.grad
  model.x.grad.fill_(0)

### `optim` class

PyTorch defines a class for optimizers, which is responsible for updating the parameters of the model. For example, the below two code blocks are equivalent:

In [ ]:
x_init = -4.
x = torch.tensor(x_init)

x = nn.Parameter(x)
L = loss(x)

L.backward()
x.data += - learning_rate * x.grad
x.grad.fill_(0)
print (f'new x: {x}, new x gradient: {x.grad}')

In [ ]:
x_init = -4.
x = torch.tensor(x_init)
x = nn.Parameter(x)

parameters = [x,]
optimizer = torch.optim.SGD(parameters, lr=learning_rate)
  
L = loss(x)
  
L.backward()
optimizer.step()
optimizer.zero_grad()
print (f'new x: {x}, new x gradient: {x.grad}')

Note that SGD stands for Stochastic Gradient Descent. We usually use SGD optimizer with a random batch from the entire training set, hence the name "stochastic".

While the above two code blocks are of roughtly the same size (the optimizer one is actually longer), when we have millions of parameters, the second code block would relieve us from the burden of writing millions of parameter update equations. For example, let's look at a case with three parameters.

In [ ]:
def loss_3params(x, y, z):
  return (x - 2) ** 2 + (y - x) ** 2 + (z - y) ** 2

In [ ]:
x_init = -4.
x = torch.tensor(x_init)
x = nn.Parameter(x)

y_init = -3.
y = torch.tensor(y_init)
y = nn.Parameter(y)

z_init = -2.
z = torch.tensor(z_init)
z = nn.Parameter(z)

L = loss_3params(x, y, z)

L.backward()

x.data += - learning_rate * x.grad
x.grad.fill_(0)
y.data += - learning_rate * y.grad
y.grad.fill_(0)
z.data += - learning_rate * z.grad
z.grad.fill_(0)
print (f'new x: {x}, new y: {y}, new z: {z}')

In [ ]:
x_init = -4.
x = torch.tensor(x_init)
x = nn.Parameter(x)

y_init = -3.
y = torch.tensor(y_init)
y = nn.Parameter(y)

z_init = -2.
z = torch.tensor(z_init)
z = nn.Parameter(z)

parameters = [x, y, z]
optimizer = torch.optim.SGD(parameters, lr=learning_rate)
  
L = loss_3params(x, y, z)
  
L.backward()
optimizer.step()
optimizer.zero_grad()
print (f'new x: {x}, new y: {y}, new z: {z}')

### Putting Everything Together

Now we can put everything together and use the PyTorchy way to create and optimize a model.

In [ ]:
class Model(nn.Module):
  # initializer, defines parameters
  def __init__(self):
    super().__init__()
    val_init = -4.
    val_tensor = torch.tensor(val_init)

    self.x = nn.Parameter(val_tensor)
    self.y = nn.Parameter(val_tensor)
    self.z = nn.Parameter(val_tensor)

  # defines the forward computation
  def forward(self):
    return (self.x - 2) ** 2 + (y - x) ** 2 + (z - y) ** 2

In [ ]:
num_iters = 20
learning_rate = 0.1

# Initialize x
model = Model()
parameters = model.parameters()
optimizer = torch.optim.SGD(parameters, lr=learning_rate)

for iter in range(num_iters):
  L = model()
  print (f'iter: {iter}, loss: {L}, x: {model.x}, y: {model.y}, z: {model.z}')
  L.backward()
  optimizer.step()
  optimizer.zero_grad()

## Predefined Modules in PyTorch

PyTorch has predefined some `nn.Module` classes for us, such as a linear function (in deep learning terms, a linear layer), or a sigmoid function (a nonlinear activation function).


In [ ]:
linear_layer = nn.Linear(in_features=3, out_features=1)

print (list(linear_layer.named_parameters()))

Check out PyTorch code here: https://github.com/pytorch/pytorch/blob/master/torch/nn/modules/linear.py

In [ ]:
in_features = torch.tensor([1., 2., 3.])
print (linear_layer(in_features))

We can demonstrate its equivalence to using low-level tensor operations.

In [ ]:
weight_tensor = linear_layer.weight
bias_tensor = linear_layer.bias

output = (weight_tensor * in_features).sum() + bias_tensor
print (output)

Let's also try a nonlinear sigmoid layer.

In [ ]:
sigmoid_layer = nn.Sigmoid()
print (sigmoid_layer(in_features))

A very useful property of a `nn.Module` class is that it will collect all parameters of its members in its own parameters, as shown below.

In [ ]:
class Model(nn.Module):
  # initializer, defines parameters
  def __init__(self):
    super().__init__()
    
    self.linear_1 = nn.Linear(3, 4)
    self.linear_2 = nn.Linear(4, 4)
    self.linear_3 = nn.Linear(4, 1)
    self.sigmoid = nn.Sigmoid()

  # defines the forward computation
  def forward(self, x):
    return self.linear_3(self.linear_2(self.sigmoid(self.linear_1(x))))

In [ ]:
model = Model()
print (list(model.named_parameters()))

In [ ]:
in_features = torch.tensor([1., 2., 3.])
output = model(in_features)
print (output)

Let's look at its computation graph. Remember that all these PyTorch modules are just wrappers for the basic tensor operations.

In [ ]:
make_dot(output, params=dict(model.named_parameters()))

**Student Question:** Can you implement a gradient-descent based linear regression algorithm in PyTorch now? What should be the loss function?

# End of Section 1